# ORA
the general and specfic for KEGG, REACTOME and GO

In [1]:
import sys
sys.path.insert(0, "/Users/yangxu/PyPathway")

In [4]:
import pandas as pd
from pypathway import Reactome, GO, KEGG, ORA
from pypathway import ColorectalCancer, IdMapping, GMTUtils
from pypathway import EnrichmentExport
import os
import sys

In [3]:
# load a gmt file.
gmt = GMTUtils.parse_gmt_file("../../pypathway/tests/gmt_file/h.all.v6.0.entrez.gmt")

In [5]:
# load the example
c = ColorectalCancer()

In [6]:
# infomation of datasets
len(c.deg_list), len(c.background)

(5320, 17216)

In [7]:
res_h = ORA.run(c.deg_list, c.background, gmt)

In [15]:
res_h.table.sort_values("fdr").iloc[0][-1]

7.3846435055285931e-24

In [8]:
# inline bar plot.
res_h.plot()

## KEGG

In [16]:
# kegg enrichment for certain organism
r_kg = KEGG.run(c.deg_list, c.background, 'hsa')

In [20]:
r_kg.table.sort_values("fdr").iloc[0][-1]

2.8162417135395389e-18

In [11]:
r_kg.plot()

## Reactome

In [12]:
# the Example of using the warpper of Reactome gene set enrichment analysis

In [7]:
sybs = [x[1][0] for x in IdMapping.convert(input_id=c.deg_list, species='hsa', source='ENTREZID', target='SYMBOL') if x[1]]

In [8]:
sybs[:10]

['A2M',
 'MKKS',
 'S100A3',
 'ANKRD29',
 'TMEM250',
 'NAT1',
 'NAT2',
 'SERPINA3',
 'AAMP',
 'AARS']

In [9]:
# the input is a list of symbol
r = Reactome.run(sybs[:10], organism='Homo sapiens')

In [10]:
# the result
r.table.head()

,name,dbId,found,p-value,fdr,species
0,Acetylation,156582,2,0.000557,0.023960,Homo sapiens
1,Defective SLC6A2 causes orthostatic intoleranc...,5619109,1,0.006418,0.131517,Homo sapiens
2,Amino acid transport across the plasma membrane,352230,2,0.009394,0.131517,Homo sapiens
3,Neurotransmitter uptake and metabolism In glia...,112313,1,0.021240,0.162962,Homo sapiens
4,Astrocytic Glutamate-Glutamine Uptake And Meta...,210455,1,0.021240,0.162962,Homo sapiens


## Gene ontology

In [21]:
# make the association file using the id_mapping function
# detail using will be shown in the utils section
r = IdMapping.convert_to_dict(input_id=c.background, source='ENTREZID', target="GO", species='hsa')

In [22]:
# run go enrichment analysis via goatools 
# the inputs of study, pop, and assoc is list, list,  dict.
# the path is the folder of go obo file
# the path should be a valid filesystem path
path = os.getcwd() + "/go.obo"
rg = GO.run([str(x) for x in c.deg_list], [str(x) for x in c.background], r, obo=path)

load obo file /Users/yangxu/PyPathway/examples/analysis/go.obo
/Users/yangxu/PyPathway/examples/analysis/go.obo: fmt(1.2) rel(2017-04-28) 48,589 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..
goids not found: {'GO:0140059', 'GO:0061897', 'GO:0110026', 'GO:0140030', 'GO:0106046', 'GO:0140058', 'GO:0102798', 'GO:0061885', 'GO:0102200', 'GO:0102487', 'GO:0102500', 'GO:0061898', 'GO:0102489', 'GO:0102953', 'GO:0102568', 'GO:0102158', 'GO:0140066', 'GO:0061890', 'GO:0102773', 'GO:0140068', 'GO:0106028', 'GO:0110053', 'GO:0102389', 'GO:0106030', 'GO:0061881', 'GO:0061931', 'GO:0140052', 'GO:0120053', 'GO:0102420', 'GO:0103067', 'GO:0102756', 'GO:0102250', 'GO:0120103', 'GO:0102524', 'GO:0106045', 'GO:0061920', 'GO:0102752', 'GO:0102499', 'GO:0102751', 'GO:0102486', 'GO:0061921', 'GO:0140036', 'GO:0120072', 'GO:0120092', 'GO:0140067', 'GO:0102966', 'GO:0102797', 'GO:0102707', 'GO:0102488', 'GO:0102113', 'GO:0110024', 'GO:0140031', 'GO:0103045', 'GO:0061944', 'GO:0102294', 'GO:0102102', 'GO:0061896', 'GO:0140069', 'GO:0140065', 'GO:0102965', 'GO:0140080', 'GO:0102769', 'GO:0106022', 'GO:0102485', 'GO:0102148', 'GO:0061899', 'GO:0120058', 'GO:01

15,292 out of 17,216 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
 5,114 out of  5,320 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  15,947 GO terms are associated with 5,114 of 5,320 study items
  21,008 GO terms are associated with 15,292 of 17,216 population items


In [24]:
rg.table.head()

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,hit
0,GO:0008150,BP,e,biological_process,4755/5320,14017/17216,5.100000e-78,n.a.,4755,1.070000e-73,1.040000e-73,1.070000e-73,"10, 1000, 10000, 10005, 10006, 10008, 10013, 1..."
1,GO:0044699,BP,e,single-organism process,3375/5320,9267/17216,8.830000e-65,n.a.,3375,1.860000e-60,1.810000e-60,1.860000e-60,"1000, 10000, 10005, 10006, 10008, 10013, 10013..."
2,GO:0044763,BP,e,single-organism cellular process,2954/5320,7926/17216,1.740000e-62,n.a.,2954,3.660000e-58,3.570000e-58,3.660000e-58,"1000, 10000, 10005, 10006, 10008, 10013, 10013..."
3,GO:0009987,BP,e,cellular process,4204/5320,12197/17216,3.790000e-58,n.a.,4204,7.970000e-54,7.770000e-54,7.960000e-54,"10, 1000, 10000, 10005, 10006, 10008, 10013, 1..."
4,GO:0044710,BP,e,single-organism metabolic process,1318/5320,3082/17216,2.230000e-53,n.a.,1318,4.680000e-49,4.570000e-49,4.680000e-49,"10005, 100131187, 100133941, 10044, 10045, 100..."


In [28]:
rg.table.sort_values("p_bonferroni").iloc[0][-2]

4.7299999999999981e-102

In [8]:
# the file input of study, pop and assoc
# this function is the warpper of the Goatools
# Github: https://github.com/tanghaibao/goatools
# cite: Haibao Tang et al. (2015). GOATOOLS: Tools for Gene Ontology. Zenodo. 10.5281/zenodo.31628.
path = "../../pypathway/tests/data/"
rg = GO.run(path + 'study', path + 'population', path + 'association',
        obo=os.getcwd() + 'go-basic.obo')

obo file not found, start to download
load obo file /Users/yangxu/PyPathway/examples/analysisgo-basic.obo
/Users/yangxu/PyPathway/examples/analysisgo-basic.obo: fmt(1.2) rel(2017-11-22) 49,201 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..
goids not found: {'GO:0022625', 'GO:0022627'}


31,855 out of 33,239 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
   269 out of    276 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  788 GO terms are associated with 269 of 276 study items
  6,118 GO terms are associated with 31,855 of 33,239 population items


In [9]:
rg.table.head()

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm,hit
0,GO:0036211,BP,e,protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0504,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."
1,GO:0006464,BP,e,cellular protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0504,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."
2,GO:0006468,BP,e,protein phosphorylation,22/276,922/33239,0.000011,n.a.,22,0.0660,0.0643,0.0659,"AT1G66980, AT2G29220, AT2G41140, AT2G41970, AT..."
3,GO:0016310,BP,e,phosphorylation,22/276,996/33239,0.000035,n.a.,22,0.2120,0.2070,0.2120,"AT1G66980, AT2G29220, AT2G41140, AT2G41970, AT..."
4,GO:0043412,BP,e,macromolecule modification,33/276,1877/33239,0.000057,n.a.,33,0.3510,0.3420,0.3500,"AT1G13580, AT1G66610, AT1G66860, AT1G66980, AT..."


In [10]:
# the interactive graph display the significance of the result by color
rg.graph()

In [11]:
# test export
c = EnrichmentExport.export([rg, rg])